In [1]:
import h5py  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [2]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
#from keras.utils import np_utilsy
from keras import backend as K
import os
import gc
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from keras.layers import Concatenate
from keras.models import Model

In [3]:
#If using GPU.
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
#read database.
f = h5py.File('GWdatabase.h5','r')   
f.keys()   

<KeysViewHDF5 ['reduced_data', 'waveforms', 'yeofrho']>

In [5]:
#create a list that contains all the failure cases.
fail_num = []
index = 0
for item in f['reduced_data']['tbounce(s)']:
    if(item == -1):
        fail_num.append(index)
    index += 1
fail_case = []
for index in fail_num:
    fail_case.append([f['reduced_data']['A(km)'][index],f['reduced_data']['omega_0(rad|s)'][index],f['reduced_data']['EOS'][index]])
fail_list= []
for item in fail_case:
    tmp = str(item[2]).split("b'")[1].split("'")[0]
    tmp = "A" + str(int(item[0])) + "w" + str(item[1]) + "0_" + tmp
    fail_list.append(tmp)    

In [6]:
A_num = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        #if(float(item.split('_')[0].split('w')[0].split('A')[1]) == 300):
        if(str(item.split('_')[1]) != 'LS180' and str(item.split('_')[1]) != 'LS220' and 
        str(item.split('_')[1]) != 'LS375'):
            A_num.append(index)
    index+=1
len(A_num)

1482

In [7]:
#get the label of w.
y_train = []
y_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            y_train.append(float(item.split('_')[0].split('w')[1]))
        else:
            y_test.append(float(item.split('_')[0].split('w')[1]))
    index += 1

In [8]:
#read image data.
from PIL import Image
import numpy as np
X_train = []
X_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            #for j in range(5):
                #title = 'Final_EOS/'+ str(index) + "_" + str(j) + '.jpeg' 
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_train.append(np.array(image))
        else:
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_test.append(np.array(image))
    index += 1

In [9]:
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)

In [13]:
#using one hot to encode the label.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

In [14]:
#our model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='softmax'))
#model.add(Dense(1, activation='relu'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
#optimizer = keras.optimizers.Adam(learning_rate=0.000003)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
#model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [15]:
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=120, batch_size=16)

Epoch 1/120
93/93 [==============================] - 10s 74ms/step - loss: 7.1902 - acc: 0.0366 - val_loss: 3.4280 - val_acc: 0.0851
Epoch 2/120
93/93 [==============================] - 5s 59ms/step - loss: 3.5546 - acc: 0.0336 - val_loss: 3.3465 - val_acc: 0.0532
Epoch 3/120
93/93 [==============================] - 5s 59ms/step - loss: 3.4635 - acc: 0.0506 - val_loss: 3.3385 - val_acc: 0.0532
Epoch 4/120
93/93 [==============================] - 6s 59ms/step - loss: 3.3849 - acc: 0.0582 - val_loss: 3.3104 - val_acc: 0.0532
Epoch 5/120
93/93 [==============================] - 6s 60ms/step - loss: 3.3450 - acc: 0.0541 - val_loss: 3.2987 - val_acc: 0.0816
Epoch 6/120
93/93 [==============================] - 6s 60ms/step - loss: 3.3499 - acc: 0.0464 - val_loss: 3.2666 - val_acc: 0.0532
Epoch 7/120
93/93 [==============================] - 5s 59ms/step - loss: 3.2952 - acc: 0.0609 - val_loss: 3.1652 - val_acc: 0.0674
Epoch 8/120
93/93 [==============================] - 6s 59ms/step - loss: 3

93/93 [==============================] - 5s 59ms/step - loss: 0.6465 - acc: 0.7568 - val_loss: 1.3411 - val_acc: 0.5071
Epoch 63/120
93/93 [==============================] - 6s 59ms/step - loss: 0.6427 - acc: 0.7476 - val_loss: 1.3905 - val_acc: 0.4858
Epoch 64/120
93/93 [==============================] - 6s 60ms/step - loss: 0.5914 - acc: 0.7847 - val_loss: 1.2668 - val_acc: 0.5319
Epoch 65/120
93/93 [==============================] - 5s 57ms/step - loss: 0.5577 - acc: 0.8041 - val_loss: 1.3021 - val_acc: 0.4965
Epoch 66/120
93/93 [==============================] - 5s 58ms/step - loss: 0.5419 - acc: 0.8074 - val_loss: 1.2311 - val_acc: 0.5177
Epoch 67/120
93/93 [==============================] - 5s 59ms/step - loss: 0.4876 - acc: 0.8410 - val_loss: 1.2885 - val_acc: 0.5248
Epoch 68/120
93/93 [==============================] - 5s 59ms/step - loss: 0.5277 - acc: 0.7966 - val_loss: 1.3449 - val_acc: 0.5142
Epoch 69/120
93/93 [==============================] - 5s 59ms/step - loss: 0.4933 

In [16]:
A_num = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(str(item.split('_')[1]) != 'GShenFSU1.7' and str(item.split('_')[1]) != 'GShenFSU2.1' and 
        str(item.split('_')[1]) != 'GShenNL3'):
            A_num.append(index)
    index+=1
#get the label of w.
y_train = []
y_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            y_train.append(float(item.split('_')[0].split('w')[1]))
        else:
            y_test.append(float(item.split('_')[0].split('w')[1]))
    index += 1
#read image data.
from PIL import Image
import numpy as np
X_train = []
X_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            #for j in range(5):
                #title = 'Final_EOS/'+ str(index) + "_" + str(j) + '.jpeg' 
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_train.append(np.array(image))
        else:
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_test.append(np.array(image))
    index += 1
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)
#using one hot to encode the label.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)
#our model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='softmax'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=120, batch_size=16)

Epoch 1/120
95/95 [==============================] - 7s 66ms/step - loss: 6.6286 - acc: 0.0299 - val_loss: 3.3804 - val_acc: 0.0781
Epoch 2/120
95/95 [==============================] - 5s 58ms/step - loss: 3.5126 - acc: 0.0462 - val_loss: 3.3492 - val_acc: 0.0586
Epoch 3/120
95/95 [==============================] - 5s 58ms/step - loss: 3.4319 - acc: 0.0464 - val_loss: 3.3114 - val_acc: 0.0586
Epoch 4/120
95/95 [==============================] - 6s 58ms/step - loss: 3.3594 - acc: 0.0683 - val_loss: 3.3000 - val_acc: 0.0469
Epoch 5/120
95/95 [==============================] - 6s 60ms/step - loss: 3.3732 - acc: 0.0341 - val_loss: 3.2759 - val_acc: 0.1250
Epoch 6/120
95/95 [==============================] - 6s 59ms/step - loss: 3.3522 - acc: 0.0661 - val_loss: 3.2573 - val_acc: 0.0820
Epoch 7/120
95/95 [==============================] - 6s 59ms/step - loss: 3.2925 - acc: 0.0602 - val_loss: 3.1764 - val_acc: 0.0586
Epoch 8/120
95/95 [==============================] - 6s 59ms/step - loss: 3.

95/95 [==============================] - 5s 58ms/step - loss: 0.5176 - acc: 0.8153 - val_loss: 0.7690 - val_acc: 0.6797
Epoch 63/120
95/95 [==============================] - 5s 58ms/step - loss: 0.5277 - acc: 0.8198 - val_loss: 0.7551 - val_acc: 0.6953
Epoch 64/120
95/95 [==============================] - 5s 58ms/step - loss: 0.4791 - acc: 0.8340 - val_loss: 0.7108 - val_acc: 0.7227
Epoch 65/120
95/95 [==============================] - 5s 58ms/step - loss: 0.4771 - acc: 0.8378 - val_loss: 0.6952 - val_acc: 0.7070
Epoch 66/120
95/95 [==============================] - 5s 58ms/step - loss: 0.4471 - acc: 0.8536 - val_loss: 0.7714 - val_acc: 0.6484
Epoch 67/120
95/95 [==============================] - 5s 58ms/step - loss: 0.4429 - acc: 0.8458 - val_loss: 0.7096 - val_acc: 0.7227
Epoch 68/120
95/95 [==============================] - 5s 58ms/step - loss: 0.3978 - acc: 0.8718 - val_loss: 0.7473 - val_acc: 0.6758
Epoch 69/120
95/95 [==============================] - 5s 58ms/step - loss: 0.4338 

In [17]:
A_num = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        #if(float(item.split('_')[0].split('w')[0].split('A')[1]) == 300):
        if(str(item.split('_')[1]) != 'HShen' and str(item.split('_')[1]) != 'HShenH'):
            A_num.append(index)
    index+=1
#get the label of w.
y_train = []
y_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            y_train.append(float(item.split('_')[0].split('w')[1]))
        else:
            y_test.append(float(item.split('_')[0].split('w')[1]))
    index += 1
#read image data.
from PIL import Image
import numpy as np
X_train = []
X_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            #for j in range(5):
                #title = 'Final_EOS/'+ str(index) + "_" + str(j) + '.jpeg' 
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_train.append(np.array(image))
        else:
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_test.append(np.array(image))
    index += 1
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)
#using one hot to encode the label.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)
#our model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='softmax'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=120, batch_size=16)

Epoch 1/120
98/98 [==============================] - 6s 58ms/step - loss: 5.8329 - acc: 0.0501 - val_loss: 3.3947 - val_acc: 0.0510
Epoch 2/120
98/98 [==============================] - 6s 57ms/step - loss: 3.4742 - acc: 0.0449 - val_loss: 3.3427 - val_acc: 0.0510
Epoch 3/120
98/98 [==============================] - 6s 58ms/step - loss: 3.4401 - acc: 0.0666 - val_loss: 3.3370 - val_acc: 0.0969
Epoch 4/120
98/98 [==============================] - 6s 57ms/step - loss: 3.3940 - acc: 0.0520 - val_loss: 3.2842 - val_acc: 0.0867
Epoch 5/120
98/98 [==============================] - 6s 57ms/step - loss: 3.3301 - acc: 0.0573 - val_loss: 3.2373 - val_acc: 0.0510
Epoch 6/120
98/98 [==============================] - 6s 57ms/step - loss: 3.3116 - acc: 0.0772 - val_loss: 3.1545 - val_acc: 0.0867
Epoch 7/120
98/98 [==============================] - 6s 57ms/step - loss: 3.1769 - acc: 0.1003 - val_loss: 3.0291 - val_acc: 0.0816
Epoch 8/120
98/98 [==============================] - 6s 58ms/step - loss: 3.

98/98 [==============================] - 6s 57ms/step - loss: 0.4374 - acc: 0.8534 - val_loss: 0.7146 - val_acc: 0.7653
Epoch 63/120
98/98 [==============================] - 6s 57ms/step - loss: 0.4108 - acc: 0.8586 - val_loss: 0.7042 - val_acc: 0.7755
Epoch 64/120
98/98 [==============================] - 6s 57ms/step - loss: 0.4252 - acc: 0.8567 - val_loss: 0.7242 - val_acc: 0.7194
Epoch 65/120
98/98 [==============================] - 6s 57ms/step - loss: 0.4207 - acc: 0.8614 - val_loss: 0.7299 - val_acc: 0.7296
Epoch 66/120
98/98 [==============================] - 6s 57ms/step - loss: 0.4020 - acc: 0.8727 - val_loss: 0.6450 - val_acc: 0.7347
Epoch 67/120
98/98 [==============================] - 6s 58ms/step - loss: 0.3908 - acc: 0.8651 - val_loss: 0.6791 - val_acc: 0.7449
Epoch 68/120
98/98 [==============================] - 6s 57ms/step - loss: 0.3636 - acc: 0.8674 - val_loss: 0.6776 - val_acc: 0.7704
Epoch 69/120
98/98 [==============================] - 6s 57ms/step - loss: 0.3661 

In [18]:
A_num = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        #if(float(item.split('_')[0].split('w')[0].split('A')[1]) == 300):
        if(str(item.split('_')[1]) != 'HSDD2' and str(item.split('_')[1]) != 'HSFSG' and
          str(item.split('_')[1]) != 'HSIUF' and str(item.split('_')[1]) != 'HSNL3' and
          str(item.split('_')[1]) != 'HSTM1' and str(item.split('_')[1]) != 'HSTMA' and
          str(item.split('_')[1]) != 'SFHo' and str(item.split('_')[1]) != 'SFHx' and
          str(item.split('_')[1]) != 'BHBL' and str(item.split('_')[1]) != 'BHBLP'):
            A_num.append(index)
    index+=1
#get the label of w.
y_train = []
y_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            y_train.append(float(item.split('_')[0].split('w')[1]))
        else:
            y_test.append(float(item.split('_')[0].split('w')[1]))
    index += 1
#read image data.
from PIL import Image
import numpy as np
X_train = []
X_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            #for j in range(5):
                #title = 'Final_EOS/'+ str(index) + "_" + str(j) + '.jpeg' 
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_train.append(np.array(image))
        else:
            title = 'Final_tbounce/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_test.append(np.array(image))
    index += 1
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)
#using one hot to encode the label.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)
#our model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='softmax'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=120, batch_size=16)

Epoch 1/120
46/46 [==============================] - 6s 132ms/step - loss: 7.7074 - acc: 0.0285 - val_loss: 3.3865 - val_acc: 0.0515
Epoch 2/120
46/46 [==============================] - 4s 85ms/step - loss: 3.7944 - acc: 0.0453 - val_loss: 3.3666 - val_acc: 0.0515
Epoch 3/120
46/46 [==============================] - 4s 85ms/step - loss: 3.5371 - acc: 0.0466 - val_loss: 3.3315 - val_acc: 0.0515
Epoch 4/120
46/46 [==============================] - 4s 85ms/step - loss: 3.4397 - acc: 0.0573 - val_loss: 3.3091 - val_acc: 0.0515
Epoch 5/120
46/46 [==============================] - 4s 85ms/step - loss: 3.3983 - acc: 0.0462 - val_loss: 3.3094 - val_acc: 0.0515
Epoch 6/120
46/46 [==============================] - 4s 85ms/step - loss: 3.4057 - acc: 0.0585 - val_loss: 3.2820 - val_acc: 0.1029
Epoch 7/120
46/46 [==============================] - 4s 85ms/step - loss: 3.3126 - acc: 0.0716 - val_loss: 3.2347 - val_acc: 0.0961
Epoch 8/120
46/46 [==============================] - 4s 85ms/step - loss: 3

46/46 [==============================] - 4s 85ms/step - loss: 1.1289 - acc: 0.5813 - val_loss: 1.4829 - val_acc: 0.4544
Epoch 63/120
46/46 [==============================] - 4s 85ms/step - loss: 1.1468 - acc: 0.5552 - val_loss: 1.4481 - val_acc: 0.4456
Epoch 64/120
46/46 [==============================] - 4s 85ms/step - loss: 1.0391 - acc: 0.6055 - val_loss: 1.4570 - val_acc: 0.4466
Epoch 65/120
46/46 [==============================] - 4s 85ms/step - loss: 1.0311 - acc: 0.6336 - val_loss: 1.3673 - val_acc: 0.4903
Epoch 66/120
46/46 [==============================] - 4s 85ms/step - loss: 0.9834 - acc: 0.6304 - val_loss: 1.3625 - val_acc: 0.4942
Epoch 67/120
46/46 [==============================] - 4s 85ms/step - loss: 0.9614 - acc: 0.6457 - val_loss: 1.3987 - val_acc: 0.4709
Epoch 68/120
46/46 [==============================] - 4s 85ms/step - loss: 0.9732 - acc: 0.6452 - val_loss: 1.3653 - val_acc: 0.4903
Epoch 69/120
46/46 [==============================] - 4s 85ms/step - loss: 0.9380 

In [19]:
history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=30, batch_size=16)

Epoch 1/30
46/46 [==============================] - 4s 86ms/step - loss: 0.2684 - acc: 0.9237 - val_loss: 1.4175 - val_acc: 0.5252
Epoch 2/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2951 - acc: 0.9101 - val_loss: 1.3319 - val_acc: 0.5534
Epoch 3/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2817 - acc: 0.9169 - val_loss: 1.3697 - val_acc: 0.5340
Epoch 4/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2502 - acc: 0.9223 - val_loss: 1.3823 - val_acc: 0.5223
Epoch 5/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2541 - acc: 0.9305 - val_loss: 1.4684 - val_acc: 0.5039
Epoch 6/30
46/46 [==============================] - 4s 86ms/step - loss: 0.2619 - acc: 0.9155 - val_loss: 1.2990 - val_acc: 0.5680
Epoch 7/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2556 - acc: 0.9237 - val_loss: 1.3244 - val_acc: 0.5252
Epoch 8/30
46/46 [==============================] - 4s 85ms/step - loss: 0.2538 - a